Dr. Jesús Martí Gavilá. Departamento de Ingeniería Cartográfica, Geodesia y fotogrametría
<font color="blue" ><h1 align=center>PRÁCTICA 7</h1> <p> <h1 align=center> CURSO BÁSICO PYTHON: FICHERO GPX</h1></font>

<a href="http://www.upv.es/titulaciones/GTI/indexc.html"><img src = "http://www.upv.es/imagenes/marcaUPVN1.png" width =200 ></a>

<h2 align=center>Grado Tecnologías Interactivas | Tecnologías de la Información Geográfica</h2>



# TAREA
<div class="alert alert-block alert-info" style="margin-top: 20px">

<b>Realizar un análisis de rendimiento a partir de un fichero GNSS con waypoints en formato GPX.</b>
<p>
    
<h3>El resultado a obtener sera:</h3>  <p> 
   <li>
       <a>Un Dataframe en formato HTML con los siguientes campos:<p><p>
'Punto','Latitud','Longitud', 'Cota (m)', 'Hora', 'Distancia (m)','Dist_O (m)', 'Pendiente (%)', 'Velocidad (km/h)','Orientación (º)','Tiempo'</a><p>
   </li>
        
   <li>
       <a>Gráfico con los valores de distancia en el eje X y valores de pendiente y velocidad en la Y</a>
   </li>

   <li>
       <a>Mapa interactivo</a>
   </li>
</div>

# Estructura GPX e importación de librerías

In [1]:

"""
Creado Viernes 14 Oct 18:22:39 2022, @autor/es: Ignacio Yuste Aznar
FORMATO EJEMPLO GPX
<wpt lat="38.968484" lon="-0.148688">
	<ele>-6.847729</ele>
	<time>2020-10-06T15:42:19Z</time>
	<name>001</name>
	<sym>Flag, Blue</sym>
</wpt>
"""
import pandas as pd
import xml.etree.cElementTree as et
import os
import math as m
from datetime import datetime
import matplotlib.pyplot as plt

# Fichero de entrada y Dataframe de salida

In [ ]:
# Introducción de datos. Solicitar al usuario el nombre del fichero sin extensión.
nombre = input ("Nombre del fichero")
print (f"{nombre}")
# Este nombre también se utilizará para crear el fichero html
fichero_html = path + "/" + nombre + ".html"


In [ ]:
# Accedemos al XML y creamos un Dataframe con los campos arriba indicados
docxml = et.parse(pathgpx)
raiz = docxml.getroot()
puntos = len(raiz)-1
columnas = ['Punto','Latitud','Longitud', 'Cota (m)', 'Hora', 'Distancia (m)','Dist_O (m)', 'Pendiente (%)', 'Velocidad (km/h)','Orientación (º)','Tiempo']
df = pd.DataFrame(columns=columnas)

# Funciones de cálculo

In [ ]:
# Cálculo de la distancia parcial . función distacias
def distancias (lat1, lat2, lon1, lon2):
    diflong = abs(lon2 - lon1)
    d = m.acos((m.sin (lat1)*m.sin(lat2)) + (m.cos(lat1)*m.cos(lat2)*m.cos(diflong)))
    dr = d* 111.11
    return d, dr


In [ ]:
# Cálculo del acimut. Verificar la posición del punto B. funcion orientación
def orientacion (lat1, lat2, lon1, lon2, d):
    orient = m.atan((m.sin(lat2)-m.sin(lat1)*m.cos(d))/(m.cos(lat1)*m.sin(d)))
    return orient




In [ ]:
# Cálculo de la pendiente. En % y angulo. funcion pendiente
def pendiente (ele1,ele2,dr):
    desnivel = ele2-ele1
    Pporcentaje = (desnivel/dr)*100
    Pang = atan(desnivel/dr)
    dgeometrica = 0 #no me da tiempo a calcularla
    return Pang, Pporcentaje, dgeometrica




In [ ]:
# Cálculo de la velocidad en km/h. funcion velocidad
def velocidad (dr ,time1,time2):
    t_total=time2-time1
    v_total=dr/t_total
    return t_total,v_total


# Obtención de los datos de entrada y llamada a funciones

In [ ]:
waypoints = range (1,puntos,1)
n = 0.00
for i in waypoints:
    # Datos del punto A
    wpt1 = raiz[i]
    lat1_g = float(wpt1.get("lat"))
    lon1_g = float(wpt1.get("lon"))
    lat1 = m.radians (lat1_g)
    lon1 = m.radians (lon1_g)
    ele1 = round(float(wpt1[0].text),3)
    time1 = wpt1[1].text
    if i == 1:
        df_1 = pd.DataFrame([[i,lat1_g,lon1_g,ele1,time1,n,n,n,n,n,n]],columns = list(columnas))
        df = pd.concat([df if not df.empty else None,df_1],ignore_index=True)  #controla si un dataframe esta vacio   
# Incrementamos puntos       
    i = i+1

# Datos del punto B
    wpt2 = raiz[i]
    lat2_g = float(wpt2.get("lat"))
    lon2_g = float(wpt2.get("lon"))
    lat2 = m.radians(lat2_g)
    lon2 = m.radians(lon2_g)
    ele2 = round(float(wpt2[0].text),3)
    time2 = wpt2[1].text

# Llamar a las diferentes funciones y añadir los datos resultantes al Dataframe
    d, dr = distancias(lat1, lat2, lon1, lon2) #Introducir los parámetros necesarios para realizar el cálculo
    acimut = orientacion(lat1, lat2, lon1, lon2, d) #Introducir los parámetros necesarios para realizar el cálculo
    peng, pend, dg = pendiente(ele1,ele2,dr) #Introducir los parámetros necesarios para realizar el cálculo
    tiempo, velkmh = velocidad(dr ,time1,time2)#Introducir los parámetros necesarios para realizar el cálculo
    df_1 = pd.DataFrame([[i,lat2_g,lon2_g,ele2,time2,dg,0.00,pend,velkmh,acimut,tiempo]],columns = list(columnas))
    df = pd.concat([df,df_1],ignore_index=True )
    
    df['Dist_O (m)'] = # crear un comando para rellenar df['Dist_O (m)']

# Impresión y exportación de los obtenidos

In [ ]:
# Imprimir los valores en formato HTML
df_1.to_html(fichero_html)

In [ ]:
# Plotear el gráfico de rendimiento y guardar en JPG. Puedes ver ayuda en internet matplotlib.plot 
# Empieza para plotear 2 ejes
fig, ax = plt.subplots()
plt.plot(df["Dist_O (m)"], df['Pendiente (%)'], marker = "o", label="Pendiente %")
......................................................................................................
# indicar leyendas, titulo, etiquetas del eje X
......................................................................................................
# Exportar en JPG
......................................................................................................


# Mapa interactivo con Folium. Código ejemplo

In [ ]:
#Instalar primeramente la libreria folium
......................................................................................................

#Código creación fichero decoordenadas
df.loc[:,['Latitud','Longitud']].to_csv('coordenadas.csv',header=True, index=False) #obtienes los valores de latitud y longitud y creas un fichero de coordenadas
lista_geo = pd.read_csv('coordenadas.csv').to_numpy().tolist()

#Creación del mapa interactivo
import folium
from folium import plugins
m = folium.Map()
folium.plugins.AntPath(locations = lista_geo, popup = 'Ruta 20/11/2023', reverse="True", dash_array=[10, 20], color='red', pulse_color = 'blue').add_to(m)
m.fit_bounds(m.get_bounds())
m
